In [1]:
import sys
import os
import logging
import json
from dotenv import load_dotenv

# Change cwd to the DataEnrichment root folder
data_enrichment_root_path = os.path.abspath(os.path.join(os.getcwd(), "..", ".."))
os.chdir(data_enrichment_root_path)

if data_enrichment_root_path not in sys.path:
    sys.path.append(data_enrichment_root_path)

load_dotenv()

import db
from worker import process
from utils import download
from utils.get_video_description import get_video_description
from AudioProcessor.transcriber import transcribe_audio
from AudioProcessor.sentiment import SentimentAnalyzer
import VideoProcessor
import AudioProcessor

/Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
db_conn = db.get_connection()
video_id = "7525069106249354518"

INFO - [EnrichmentWorker] - Successfully Connected to Database


In [ ]:
video_path = os.path.abspath("./tmp/video/7525069106249354518.mp4")
audio_path = os.path.abspath("./tmp/video/7525069106249354518.wav")

transcript = """
Vă salut, dragi prieteni, vă salut, dragi români!
 Un filmuleț de aici, din Parlamentul, din Strasburg, din Parlamentul European,
 despre ce s-a întâmplat astăzi în sala de plen, în această clădire rotundă pe care o vedeți, asta de lemn, este mai rotundă.
 Dragi prieteni, astăzi, în plen, a fost un scandal.
 Un scandal pentru că partidele, Revue, Verzii și ceilalți, PSD, PNL, USR, mai departe,
 au invitat în plen, au invitat pe primarul Budapestei, exact pe primarul ăla, LGBTQ bananier,
 care a sfidat legea și ordonanța pe care domnul...
 prim-ministrul al Ungariei, Orban, a interzis parada LGBT.
 Primarul a lăsat și a acceptat această paradă de bananieri să vină în Budapesta.
 Un lucru foarte greu.
 În sală, aici, au început, cum am spus, grupul Revue, Verzii, EPP și ăștia le-au început să aplaude pe unguri, pe primarul Budapestei, așa, pe primarul Budapestei,
 și în momentul ăla, pentru că a ieșit oribil, oribil, într-o Uniune Europeană să susții LGBTQ, așa cum fac ei, este un lucru foarte grav.
 Și pentru asta, doamna Diana Ioanoviț-Oșoacă a dat tonul, a început să strige, să huiduie, să... nu să înjure, dar să strige contra primarului, alături de ea s-au alăturat și alți suveraniști, alți europaralători, alți...
 parlamentari, care susțin familia tradițională și familia compusă din bărbat și femeie, și vă dați seama că a ieșit un mic show în Parlamentul European, provocat de globaliști,
 provocat de acei extrimiști care ne fac pe noi, sau pe Pipera, pe Oșoacă, pe Luis Lazarus, pe naționaliști, îi fac extrimiști.
 Nu, extrimiștii sunt exact...
 cum își fondă line și toată armata ei, care își bat joc, pur și simplu își bat joc de cetățenii Uniunii Europene și de copiii Uniunii Europeni, cât și de pensionarii Uniunii Europene.
 Deci, este un lucru grav ce s-a întâmplat astăzi, dar ce să-i facem decât să ne spunem părerea și decât să îi huiduim înapoi?
 Mare lucru nu putem.
 Pentru că suntem noi în minoritate.
 Asta au fost aici din Parlamentul European de la Strasburg.
 Vă salut!
"""

sentiment_results = {
    'positive': 0.07932209223508835,
    'neutral': 0.0,
    'negative': 0.9206779599189758
 }

description = """
primarul din Budapesta huiduit de suveranisti in plenul de la Strasbourg #Romania #luislazarus #dianasosoaca #suveranitate #bananierii
"""

In [3]:
video_url, video_path, audio_path = download.tiktok_full(video_id)


INFO - [EnrichmentWorker] - Downloading MP4 for 7525069106249354518 → /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/video/7525069106249354518.mp4
INFO - [EnrichmentWorker] - Successfully extracted audio to: /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/audio/7525069106249354518.wav


In [4]:
model_name = "ggml-large-v3.bin" 
transcript, lang, output = transcribe_audio(audio_path, model_name)
logging.info(transcript)

INFO - [EnrichmentWorker] - Starting transcriber model ggml-large-v3.bin for audio in /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/audio/7525069106249354518.wav
INFO - [EnrichmentWorker] - Vă salut, dragi prieteni, vă salut, dragi români!
 Un filmuleț de aici, din Parlamentul, din Strasburg, din Parlamentul European,
 despre ce s-a întâmplat astăzi în sala de plen, în această clădire rotundă pe care o vedeți, asta de lemn, este mai rotundă.
 Dragi prieteni, astăzi, în plen, a fost un scandal.
 Un scandal pentru că partidele, Revue, Verzii și ceilalți, PSD, PNL, USR, mai departe,
 au invitat în plen, au invitat pe primarul Budapestei, exact pe primarul ăla, LGBTQ bananier,
 care a sfidat legea și ordonanța pe care domnul...
 prim-ministrul al Ungariei, Orban, a interzis parada LGBT.
 Primarul a lăsat și a acceptat această paradă de bananieri să vină în Budapesta.
 Un lucru foarte greu.
 În sală, aici, au început, cum am spus, grupul Revue, Verzii, EPP și ăștia le-a

In [5]:
sentiment_results = SentimentAnalyzer.get_sentiment(transcript, "ro")
sentiment_results

INFO - [EnrichmentWorker] - Loading model DGurgurov/xlm-r_romanian_sentiment for language ro
INFO - [EnrichmentWorker] - Successfully loaded model DGurgurov/xlm-r_romanian_sentiment
/Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/venv/lib/python3.13/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `XLMRobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
INFO - [EnrichmentWorker] - SENTIMENT MODEL OUTPUT SequenceClassifierOutput(loss=None, logits=tensor([[ 1.3207, -1.1309]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
INFO - [EnrichmentWorker] - SENTIMENT MODEL SCORES [0.92067796 0.07932209]


In [3]:
description = get_video_description(video_id, db_conn)
description

In [14]:
analysis_result_json = VideoProcessor.process(video_path, transcript, sentiment_results, description)

INFO - [EnrichmentWorker] - Starting video processing for /Users/rolandteslaru/Desktop/ZeruelNet/packages/DataEnrichment/tmp/video/7525069106249354518.mp4
INFO - [EnrichmentWorker] - AFC is enabled with max remote calls: 10.
INFO - [EnrichmentWorker] - HTTP Request: POST https://aiplatform.googleapis.com/v1beta1/projects/zeruel-ney/locations/global/publishers/google/models/gemini-2.5-flash-lite:generateContent "HTTP/1.1 200 OK"
INFO - [EnrichmentWorker] - AFC remote call 1 is done.
INFO - [EnrichmentWorker] - VIDEO Analysis result {'summary': "The video features a man speaking in what appears to be the European Parliament in Strasbourg. He criticizes the presence of the Mayor of Budapest at a plenary session, referring to him as 'LGBTQ bananier' and accusing him of defying Hungarian Prime Minister Viktor Orbán's ban on an LGBT parade. The speaker alleges that European Parliament members from parties like the Greens, EPP, PSD, PNL, and USR applauded the Mayor, which he frames as a scand

In [15]:
analysis_result_json

{'summary': "The video features a man speaking in what appears to be the European Parliament in Strasbourg. He criticizes the presence of the Mayor of Budapest at a plenary session, referring to him as 'LGBTQ bananier' and accusing him of defying Hungarian Prime Minister Viktor Orbán's ban on an LGBT parade. The speaker alleges that European Parliament members from parties like the Greens, EPP, PSD, PNL, and USR applauded the Mayor, which he frames as a scandal and a sign of 'Western decay.' He contrasts this with 'traditional values' and the support for family composed of a man and a woman, implying that those who support LGBT rights are 'extremists' and 'globalists' who are undermining European citizens and pensioners. The speaker, along with Diana Iovanovici-Șoșoacă and other 'nationalists,' are portrayed as standing against this perceived agenda.",
 'identified_subjects': [{'subject': 'European Parliament', 'stance': -0.8},
  {'subject': 'Viktor Orbán', 'stance': 0.8},
  {'subject'